In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import pathlib
from datetime import datetime
import math
import sys

import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates

from sklearn import metrics
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

import xgboost as xg

%matplotlib inline

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
# df_lag = pd.read_csv('/Users/yashwanthkaruparthi/Developer/energy_demand/research/data/8-feat-eng/feat-lag.csv')

df_clean = pd.read_csv('/Users/yashwanthkaruparthi/Developer/energy_demand/resrc/clean-preprocessed_last.csv')

df_daily = pd.read_csv('/Users/yashwanthkaruparthi/Developer/energy_demand/resrc/data/7-usable/1-daily_data.csv')

In [3]:
df_clean.head()

,Unnamed: 0,temp_min,temp_max,temp_mean,temp_median,temp_max_hour,temp_min_hour,dew_point_temp_min,dew_point_temp_max,dew_point_temp_mean,dew_point_temp_median,dew_point_temp_max_hour,dew_point_temp_min_hour,rel_hum_min,rel_hum_max,rel_hum_mean,rel_hum_median,rel_hum_max_hour,rel_hum_min_hour,visibility_min,visibility_max,visibility_mean,visibility_median,visibility_max_hour,visibility_min_hour,press_min,press_max,press_mean,press_median,press_max_hour,press_min_hour,hmdxx_min,hmdxx_max,hmdxx_mean,hmdxx_median,hmdxx_max_hour,hmdxx_min_hour,sun_set,sun_rise,hourly_demand,day_of_week,week_of_year,year
0,1994-01-01,-1.8,2.8,0.575000,0.65,12.0,5.0,-4.8,1.1,-1.387500,-0.65,19.0,7.0,73.0,99.0,87.000000,86.5,14.0,9.0,0.8,40.2,17.841667,16.1,11.0,14.0,98.51,99.91,99.075833,98.840,0.0,16.0,-4.976663,0.686009,-1.884900,-1.513575,13.0,7.0,17.0,8.0,14152.0,5.0,52.0,1994.0
1,1994-01-02,-14.3,1.7,-8.704167,-9.90,0.0,23.0,-20.3,0.5,-12.445833,-13.90,0.0,23.0,60.0,92.0,74.416667,75.5,0.0,23.0,6.4,40.2,28.387500,25.0,9.0,6.0,98.77,100.51,99.920000,100.125,18.0,0.0,-19.165499,-0.338394,-12.827363,-14.393998,0.0,23.0,17.0,8.0,16527.0,6.0,52.0,1994.0
2,1994-01-03,-16.3,-10.3,-12.612500,-12.00,17.0,2.0,-23.2,-12.6,-15.820833,-15.00,17.0,2.0,55.0,91.0,77.250000,80.5,9.0,2.0,1.2,25.0,10.404167,8.0,0.0,9.0,99.41,100.47,100.057083,100.155,0.0,23.0,-21.317386,-14.555717,-17.143932,-16.423198,17.0,2.0,17.0,8.0,17550.0,0.0,1.0,1994.0
3,1994-01-04,-11.5,-7.4,-9.833333,-9.95,15.0,8.0,-15.0,-11.5,-13.570833,-13.70,13.0,0.0,57.0,84.0,74.416667,76.0,10.0,16.0,9.7,40.2,24.879167,24.1,13.0,10.0,98.27,99.34,98.675417,98.610,0.0,14.0,-15.874948,-11.706660,-14.183286,-14.310177,15.0,8.0,17.0,8.0,17395.0,1.0,1.0,1994.0
4,1994-01-05,-17.5,-7.2,-11.866667,-11.75,5.0,22.0,-20.8,-10.7,-16.154167,-17.00,4.0,22.0,56.0,84.0,70.791667,74.0,4.0,14.0,1.0,40.2,27.033333,25.0,9.0,7.0,98.44,100.37,99.352083,99.375,22.0,3.0,-22.394182,-11.256395,-16.407695,-16.464188,5.0,22.0,17.0,8.0,18485.0,2.0,1.0,1994.0


In [4]:
df_daily.tail()

,Unnamed: 0,date,temp,dew_point_temp,rel_hum,wind_speed,visibility,press,wind_chill,hourly_demand,hmdxx,temp_min,temp_max,temp_mean,temp_median,dew_point_temp_min,dew_point_temp_max,dew_point_temp_mean,dew_point_temp_median,rel_hum_min,rel_hum_max,rel_hum_mean,rel_hum_median,visibility_min,visibility_max,visibility_mean,visibility_median,press_min,press_max,press_mean,press_median,hmdxx_min,hmdxx_max,hmdxx_mean,hmdxx_median
9126,9126,2018-12-27,-2.541667,-6.575000,74.166667,17.750000,23.633333,100.791667,-9.166667,15698.625000,-5.917272,-6.9,2.5,-2.541667,-1.95,-12.7,0.0,-6.575000,-5.35,61.0,83.0,74.166667,77.0,12.9,24.1,23.633333,24.1,99.31,101.50,100.791667,101.075,-11.155717,0.336425,-5.917272,-5.212529
9127,9127,2018-12-28,8.016667,6.254167,88.833333,15.750000,17.204167,98.661667,NaN,14809.208333,7.868511,2.7,12.4,8.016667,8.35,0.4,10.8,6.254167,7.25,74.0,95.0,88.833333,91.5,3.6,24.1,17.204167,24.1,98.37,99.21,98.661667,98.635,0.738694,13.912823,7.868511,8.262797
9128,9128,2018-12-29,-2.525000,-7.520833,68.875000,19.916667,24.033333,99.903750,-9.722222,15135.791667,-6.063010,-7.0,3.6,-2.525000,-3.05,-11.4,-0.6,-7.520833,-9.35,56.0,80.0,68.875000,71.0,22.5,24.1,24.033333,24.1,98.85,100.43,99.903750,100.135,-11.125707,1.291505,-6.063010,-7.026121
9129,9129,2018-12-30,-2.404167,-5.829167,77.666667,10.958333,19.358333,99.869167,-6.150000,15326.625000,-5.735336,-6.4,0.6,-2.404167,-1.60,-10.5,-2.8,-5.829167,-5.40,68.0,88.0,77.666667,75.5,1.6,24.1,19.358333,24.1,99.65,100.15,99.869167,99.870,-10.147759,-2.448366,-5.735336,-4.982904
9130,9130,2018-12-31,1.204167,-1.645833,81.708333,10.458333,18.091667,99.302500,-3.000000,15016.208333,-1.278330,-2.0,4.3,1.204167,2.25,-6.4,3.4,-1.645833,-1.30,65.0,94.0,81.708333,82.5,4.8,24.1,18.091667,24.1,97.37,100.03,99.302500,99.695,-5.229812,3.073132,-1.278330,0.062595


In [5]:
# df_daily.set_index('date', inplace=True)

In [6]:
df_daily = df_daily.reset_index()

In [7]:
df_daily.rename(columns={'date':'datetime'}, inplace=True)

# checking duplicate columns
if df_daily.columns.duplicated().any():
    print("Warning: Duplicate columns found and will be dropped.")
    df_daily = df_daily.loc[:, ~df_daily.columns.duplicated()]


# df_daily['date'] = pd.to_datetime(df_daily['date'])


# df_daily['week_of_year'] = df_daily['date'].dt.isocalendar().week.astype(str)

# df_daily['day_of_week'] = df_daily['date'].dt.dayofweek


# if df_daily['day_of_week'].dtype != 'string':
#     df_daily['day_of_week'] = df_daily['day_of_week'].astype(str)


# df_daily['year'] = df_daily['date'].dt.year
# df_daily['month'] = df_daily['date'].dt.month
# df_daily['day'] = df_daily['date'].dt.day


# columns_to_drop = ['date']
# if 'sun_rise' in df_daily.columns:
#     columns_to_drop.append('sun_rise')
# df_daily.drop(columns=columns_to_drop, axis=1, inplace=True)


# if df_daily.columns.duplicated().any():
#     print("Warning: Duplicate columns found and will be dropped.")
#     df_daily = df_daily.loc[:, ~df_daily.columns.duplicated()]

In [8]:
df_daily.head()

,index,Unnamed: 0,datetime,temp,dew_point_temp,rel_hum,wind_speed,visibility,press,wind_chill,hourly_demand,hmdxx,temp_min,temp_max,temp_mean,temp_median,dew_point_temp_min,dew_point_temp_max,dew_point_temp_mean,dew_point_temp_median,rel_hum_min,rel_hum_max,rel_hum_mean,rel_hum_median,visibility_min,visibility_max,visibility_mean,visibility_median,press_min,press_max,press_mean,press_median,hmdxx_min,hmdxx_max,hmdxx_mean,hmdxx_median
0,0,0,1994-01-01,0.575000,-1.387500,87.000000,20.333333,17.841667,99.075833,-5.888889,14655.208333,-1.884900,-1.8,2.8,0.575000,0.65,-4.8,1.1,-1.387500,-0.65,73.0,99.0,87.000000,86.5,0.8,40.2,17.841667,16.1,98.51,99.91,99.075833,98.840,-4.976663,0.686009,-1.884900,-1.513575
1,1,1,1994-01-02,-8.704167,-12.445833,74.416667,17.875000,28.387500,99.920000,-16.681818,15783.333333,-12.827363,-14.3,1.7,-8.704167,-9.90,-20.3,0.5,-12.445833,-13.90,60.0,92.0,74.416667,75.5,6.4,40.2,28.387500,25.0,98.77,100.51,99.920000,100.125,-19.165499,-0.338394,-12.827363,-14.393998
2,2,2,1994-01-03,-12.612500,-15.820833,77.250000,17.666667,10.404167,100.057083,-20.583333,18992.708333,-17.143932,-16.3,-10.3,-12.612500,-12.00,-23.2,-12.6,-15.820833,-15.00,55.0,91.0,77.250000,80.5,1.2,25.0,10.404167,8.0,99.41,100.47,100.057083,100.155,-21.317386,-14.555717,-17.143932,-16.423198
3,3,3,1994-01-04,-9.833333,-13.570833,74.416667,25.791667,24.879167,98.675417,-18.541667,19080.708333,-14.183286,-11.5,-7.4,-9.833333,-9.95,-15.0,-11.5,-13.570833,-13.70,57.0,84.0,74.416667,76.0,9.7,40.2,24.879167,24.1,98.27,99.34,98.675417,98.610,-15.874948,-11.706660,-14.183286,-14.310177
4,4,4,1994-01-05,-11.866667,-16.154167,70.791667,22.875000,27.033333,99.352083,-20.521739,19183.208333,-16.407695,-17.5,-7.2,-11.866667,-11.75,-20.8,-10.7,-16.154167,-17.00,56.0,84.0,70.791667,74.0,1.0,40.2,27.033333,25.0,98.44,100.37,99.352083,99.375,-22.394182,-11.256395,-16.407695,-16.464188


In [9]:
def extract_date_components(df, date_col):
    if pd.api.types.is_datetime64_dtype(df_daily[date_col]):  # check if it's already datetime
        df_daily['date'] = df_daily[date_col].dt.day
        df_daily['month'] = df_daily[date_col].dt.month
        df_daily['year'] = df_daily[date_col].dt.year
    else: 
        try:
            df_daily[date_col] = pd.to_datetime(df_daily[date_col])  
            df_daily['date'] = df_daily[date_col].dt.day
            df_daily['month'] = df_daily[date_col].dt.month
            df_daily['year'] = df_daily[date_col].dt.year
        except (ValueError, pd.errors.OutOfBoundsDatetime):
            print("Error: Unable to convert '{}' to datetime format. Check for invalid date strings.".format(date_col))

    return df_daily


In [10]:
date_col = 'datetime'
df_daily = extract_date_components(df_daily.copy(), date_col)

In [11]:
df_daily.head()

,index,Unnamed: 0,datetime,temp,dew_point_temp,rel_hum,wind_speed,visibility,press,wind_chill,hourly_demand,hmdxx,temp_min,temp_max,temp_mean,temp_median,dew_point_temp_min,dew_point_temp_max,dew_point_temp_mean,dew_point_temp_median,rel_hum_min,rel_hum_max,rel_hum_mean,rel_hum_median,visibility_min,visibility_max,visibility_mean,visibility_median,press_min,press_max,press_mean,press_median,hmdxx_min,hmdxx_max,hmdxx_mean,hmdxx_median,date,month,year
0,0,0,1994-01-01,0.575000,-1.387500,87.000000,20.333333,17.841667,99.075833,-5.888889,14655.208333,-1.884900,-1.8,2.8,0.575000,0.65,-4.8,1.1,-1.387500,-0.65,73.0,99.0,87.000000,86.5,0.8,40.2,17.841667,16.1,98.51,99.91,99.075833,98.840,-4.976663,0.686009,-1.884900,-1.513575,1,1,1994
1,1,1,1994-01-02,-8.704167,-12.445833,74.416667,17.875000,28.387500,99.920000,-16.681818,15783.333333,-12.827363,-14.3,1.7,-8.704167,-9.90,-20.3,0.5,-12.445833,-13.90,60.0,92.0,74.416667,75.5,6.4,40.2,28.387500,25.0,98.77,100.51,99.920000,100.125,-19.165499,-0.338394,-12.827363,-14.393998,2,1,1994
2,2,2,1994-01-03,-12.612500,-15.820833,77.250000,17.666667,10.404167,100.057083,-20.583333,18992.708333,-17.143932,-16.3,-10.3,-12.612500,-12.00,-23.2,-12.6,-15.820833,-15.00,55.0,91.0,77.250000,80.5,1.2,25.0,10.404167,8.0,99.41,100.47,100.057083,100.155,-21.317386,-14.555717,-17.143932,-16.423198,3,1,1994
3,3,3,1994-01-04,-9.833333,-13.570833,74.416667,25.791667,24.879167,98.675417,-18.541667,19080.708333,-14.183286,-11.5,-7.4,-9.833333,-9.95,-15.0,-11.5,-13.570833,-13.70,57.0,84.0,74.416667,76.0,9.7,40.2,24.879167,24.1,98.27,99.34,98.675417,98.610,-15.874948,-11.706660,-14.183286,-14.310177,4,1,1994
4,4,4,1994-01-05,-11.866667,-16.154167,70.791667,22.875000,27.033333,99.352083,-20.521739,19183.208333,-16.407695,-17.5,-7.2,-11.866667,-11.75,-20.8,-10.7,-16.154167,-17.00,56.0,84.0,70.791667,74.0,1.0,40.2,27.033333,25.0,98.44,100.37,99.352083,99.375,-22.394182,-11.256395,-16.407695,-16.464188,5,1,1994


In [12]:
df_daily.tail()

,index,Unnamed: 0,datetime,temp,dew_point_temp,rel_hum,wind_speed,visibility,press,wind_chill,hourly_demand,hmdxx,temp_min,temp_max,temp_mean,temp_median,dew_point_temp_min,dew_point_temp_max,dew_point_temp_mean,dew_point_temp_median,rel_hum_min,rel_hum_max,rel_hum_mean,rel_hum_median,visibility_min,visibility_max,visibility_mean,visibility_median,press_min,press_max,press_mean,press_median,hmdxx_min,hmdxx_max,hmdxx_mean,hmdxx_median,date,month,year
9126,9126,9126,2018-12-27,-2.541667,-6.575000,74.166667,17.750000,23.633333,100.791667,-9.166667,15698.625000,-5.917272,-6.9,2.5,-2.541667,-1.95,-12.7,0.0,-6.575000,-5.35,61.0,83.0,74.166667,77.0,12.9,24.1,23.633333,24.1,99.31,101.50,100.791667,101.075,-11.155717,0.336425,-5.917272,-5.212529,27,12,2018
9127,9127,9127,2018-12-28,8.016667,6.254167,88.833333,15.750000,17.204167,98.661667,NaN,14809.208333,7.868511,2.7,12.4,8.016667,8.35,0.4,10.8,6.254167,7.25,74.0,95.0,88.833333,91.5,3.6,24.1,17.204167,24.1,98.37,99.21,98.661667,98.635,0.738694,13.912823,7.868511,8.262797,28,12,2018
9128,9128,9128,2018-12-29,-2.525000,-7.520833,68.875000,19.916667,24.033333,99.903750,-9.722222,15135.791667,-6.063010,-7.0,3.6,-2.525000,-3.05,-11.4,-0.6,-7.520833,-9.35,56.0,80.0,68.875000,71.0,22.5,24.1,24.033333,24.1,98.85,100.43,99.903750,100.135,-11.125707,1.291505,-6.063010,-7.026121,29,12,2018
9129,9129,9129,2018-12-30,-2.404167,-5.829167,77.666667,10.958333,19.358333,99.869167,-6.150000,15326.625000,-5.735336,-6.4,0.6,-2.404167,-1.60,-10.5,-2.8,-5.829167,-5.40,68.0,88.0,77.666667,75.5,1.6,24.1,19.358333,24.1,99.65,100.15,99.869167,99.870,-10.147759,-2.448366,-5.735336,-4.982904,30,12,2018
9130,9130,9130,2018-12-31,1.204167,-1.645833,81.708333,10.458333,18.091667,99.302500,-3.000000,15016.208333,-1.278330,-2.0,4.3,1.204167,2.25,-6.4,3.4,-1.645833,-1.30,65.0,94.0,81.708333,82.5,4.8,24.1,18.091667,24.1,97.37,100.03,99.302500,99.695,-5.229812,3.073132,-1.278330,0.062595,31,12,2018


In [13]:
columns_to_drop = ['Unnamed: 0']
if 'sun_rise' in df_daily.columns:
    columns_to_drop.append('sun_rise')
df_daily.drop(columns=columns_to_drop, axis=1, inplace=True)

if df_daily.columns.duplicated().any():
    print("Warning: Duplicate columns found and will be dropped.")
    df_daily = df_daily.loc[:, ~df_daily.columns.duplicated()]

In [14]:
df_daily.head()

,index,datetime,temp,dew_point_temp,rel_hum,wind_speed,visibility,press,wind_chill,hourly_demand,hmdxx,temp_min,temp_max,temp_mean,temp_median,dew_point_temp_min,dew_point_temp_max,dew_point_temp_mean,dew_point_temp_median,rel_hum_min,rel_hum_max,rel_hum_mean,rel_hum_median,visibility_min,visibility_max,visibility_mean,visibility_median,press_min,press_max,press_mean,press_median,hmdxx_min,hmdxx_max,hmdxx_mean,hmdxx_median,date,month,year
0,0,1994-01-01,0.575000,-1.387500,87.000000,20.333333,17.841667,99.075833,-5.888889,14655.208333,-1.884900,-1.8,2.8,0.575000,0.65,-4.8,1.1,-1.387500,-0.65,73.0,99.0,87.000000,86.5,0.8,40.2,17.841667,16.1,98.51,99.91,99.075833,98.840,-4.976663,0.686009,-1.884900,-1.513575,1,1,1994
1,1,1994-01-02,-8.704167,-12.445833,74.416667,17.875000,28.387500,99.920000,-16.681818,15783.333333,-12.827363,-14.3,1.7,-8.704167,-9.90,-20.3,0.5,-12.445833,-13.90,60.0,92.0,74.416667,75.5,6.4,40.2,28.387500,25.0,98.77,100.51,99.920000,100.125,-19.165499,-0.338394,-12.827363,-14.393998,2,1,1994
2,2,1994-01-03,-12.612500,-15.820833,77.250000,17.666667,10.404167,100.057083,-20.583333,18992.708333,-17.143932,-16.3,-10.3,-12.612500,-12.00,-23.2,-12.6,-15.820833,-15.00,55.0,91.0,77.250000,80.5,1.2,25.0,10.404167,8.0,99.41,100.47,100.057083,100.155,-21.317386,-14.555717,-17.143932,-16.423198,3,1,1994
3,3,1994-01-04,-9.833333,-13.570833,74.416667,25.791667,24.879167,98.675417,-18.541667,19080.708333,-14.183286,-11.5,-7.4,-9.833333,-9.95,-15.0,-11.5,-13.570833,-13.70,57.0,84.0,74.416667,76.0,9.7,40.2,24.879167,24.1,98.27,99.34,98.675417,98.610,-15.874948,-11.706660,-14.183286,-14.310177,4,1,1994
4,4,1994-01-05,-11.866667,-16.154167,70.791667,22.875000,27.033333,99.352083,-20.521739,19183.208333,-16.407695,-17.5,-7.2,-11.866667,-11.75,-20.8,-10.7,-16.154167,-17.00,56.0,84.0,70.791667,74.0,1.0,40.2,27.033333,25.0,98.44,100.37,99.352083,99.375,-22.394182,-11.256395,-16.407695,-16.464188,5,1,1994


In [15]:
df_daily.tail()

,index,datetime,temp,dew_point_temp,rel_hum,wind_speed,visibility,press,wind_chill,hourly_demand,hmdxx,temp_min,temp_max,temp_mean,temp_median,dew_point_temp_min,dew_point_temp_max,dew_point_temp_mean,dew_point_temp_median,rel_hum_min,rel_hum_max,rel_hum_mean,rel_hum_median,visibility_min,visibility_max,visibility_mean,visibility_median,press_min,press_max,press_mean,press_median,hmdxx_min,hmdxx_max,hmdxx_mean,hmdxx_median,date,month,year
9126,9126,2018-12-27,-2.541667,-6.575000,74.166667,17.750000,23.633333,100.791667,-9.166667,15698.625000,-5.917272,-6.9,2.5,-2.541667,-1.95,-12.7,0.0,-6.575000,-5.35,61.0,83.0,74.166667,77.0,12.9,24.1,23.633333,24.1,99.31,101.50,100.791667,101.075,-11.155717,0.336425,-5.917272,-5.212529,27,12,2018
9127,9127,2018-12-28,8.016667,6.254167,88.833333,15.750000,17.204167,98.661667,NaN,14809.208333,7.868511,2.7,12.4,8.016667,8.35,0.4,10.8,6.254167,7.25,74.0,95.0,88.833333,91.5,3.6,24.1,17.204167,24.1,98.37,99.21,98.661667,98.635,0.738694,13.912823,7.868511,8.262797,28,12,2018
9128,9128,2018-12-29,-2.525000,-7.520833,68.875000,19.916667,24.033333,99.903750,-9.722222,15135.791667,-6.063010,-7.0,3.6,-2.525000,-3.05,-11.4,-0.6,-7.520833,-9.35,56.0,80.0,68.875000,71.0,22.5,24.1,24.033333,24.1,98.85,100.43,99.903750,100.135,-11.125707,1.291505,-6.063010,-7.026121,29,12,2018
9129,9129,2018-12-30,-2.404167,-5.829167,77.666667,10.958333,19.358333,99.869167,-6.150000,15326.625000,-5.735336,-6.4,0.6,-2.404167,-1.60,-10.5,-2.8,-5.829167,-5.40,68.0,88.0,77.666667,75.5,1.6,24.1,19.358333,24.1,99.65,100.15,99.869167,99.870,-10.147759,-2.448366,-5.735336,-4.982904,30,12,2018
9130,9130,2018-12-31,1.204167,-1.645833,81.708333,10.458333,18.091667,99.302500,-3.000000,15016.208333,-1.278330,-2.0,4.3,1.204167,2.25,-6.4,3.4,-1.645833,-1.30,65.0,94.0,81.708333,82.5,4.8,24.1,18.091667,24.1,97.37,100.03,99.302500,99.695,-5.229812,3.073132,-1.278330,0.062595,31,12,2018


In [16]:
df_daily.rename(columns={'hourly_demand':'daily_demand'}, inplace=True)

In [17]:
df_daily.head()

,index,datetime,temp,dew_point_temp,rel_hum,wind_speed,visibility,press,wind_chill,daily_demand,hmdxx,temp_min,temp_max,temp_mean,temp_median,dew_point_temp_min,dew_point_temp_max,dew_point_temp_mean,dew_point_temp_median,rel_hum_min,rel_hum_max,rel_hum_mean,rel_hum_median,visibility_min,visibility_max,visibility_mean,visibility_median,press_min,press_max,press_mean,press_median,hmdxx_min,hmdxx_max,hmdxx_mean,hmdxx_median,date,month,year
0,0,1994-01-01,0.575000,-1.387500,87.000000,20.333333,17.841667,99.075833,-5.888889,14655.208333,-1.884900,-1.8,2.8,0.575000,0.65,-4.8,1.1,-1.387500,-0.65,73.0,99.0,87.000000,86.5,0.8,40.2,17.841667,16.1,98.51,99.91,99.075833,98.840,-4.976663,0.686009,-1.884900,-1.513575,1,1,1994
1,1,1994-01-02,-8.704167,-12.445833,74.416667,17.875000,28.387500,99.920000,-16.681818,15783.333333,-12.827363,-14.3,1.7,-8.704167,-9.90,-20.3,0.5,-12.445833,-13.90,60.0,92.0,74.416667,75.5,6.4,40.2,28.387500,25.0,98.77,100.51,99.920000,100.125,-19.165499,-0.338394,-12.827363,-14.393998,2,1,1994
2,2,1994-01-03,-12.612500,-15.820833,77.250000,17.666667,10.404167,100.057083,-20.583333,18992.708333,-17.143932,-16.3,-10.3,-12.612500,-12.00,-23.2,-12.6,-15.820833,-15.00,55.0,91.0,77.250000,80.5,1.2,25.0,10.404167,8.0,99.41,100.47,100.057083,100.155,-21.317386,-14.555717,-17.143932,-16.423198,3,1,1994
3,3,1994-01-04,-9.833333,-13.570833,74.416667,25.791667,24.879167,98.675417,-18.541667,19080.708333,-14.183286,-11.5,-7.4,-9.833333,-9.95,-15.0,-11.5,-13.570833,-13.70,57.0,84.0,74.416667,76.0,9.7,40.2,24.879167,24.1,98.27,99.34,98.675417,98.610,-15.874948,-11.706660,-14.183286,-14.310177,4,1,1994
4,4,1994-01-05,-11.866667,-16.154167,70.791667,22.875000,27.033333,99.352083,-20.521739,19183.208333,-16.407695,-17.5,-7.2,-11.866667,-11.75,-20.8,-10.7,-16.154167,-17.00,56.0,84.0,70.791667,74.0,1.0,40.2,27.033333,25.0,98.44,100.37,99.352083,99.375,-22.394182,-11.256395,-16.407695,-16.464188,5,1,1994


### rolling window

In [18]:
df_daily['demand_rolling_mean_3'] = df_daily['daily_demand'].rolling(window=3).mean()
df_daily['demand_rolling_std_7'] = df_daily['daily_demand'].rolling(window=7).std()

window_size = 7
df_daily['temp_roll_mean'] = df_daily['temp'].rolling(window=window_size).mean()
df_daily['temp_roll_std'] = df_daily['temp'].rolling(window=window_size).std()
df_daily['rel_hum_roll_mean'] = df_daily['rel_hum'].rolling(window=window_size).mean()
df_daily['rel_hum_roll_std'] = df_daily['rel_hum'].rolling(window=window_size).std()

In [19]:
df_daily.head(30)

,index,datetime,temp,dew_point_temp,rel_hum,wind_speed,visibility,press,wind_chill,daily_demand,hmdxx,temp_min,temp_max,temp_mean,temp_median,dew_point_temp_min,dew_point_temp_max,dew_point_temp_mean,dew_point_temp_median,rel_hum_min,rel_hum_max,rel_hum_mean,rel_hum_median,visibility_min,visibility_max,visibility_mean,visibility_median,press_min,press_max,press_mean,press_median,hmdxx_min,hmdxx_max,hmdxx_mean,hmdxx_median,date,month,year,demand_rolling_mean_3,demand_rolling_std_7,temp_roll_mean,temp_roll_std,rel_hum_roll_mean,rel_hum_roll_std
0,0,1994-01-01,0.575000,-1.387500,87.000000,20.333333,17.841667,99.075833,-5.888889,14655.208333,-1.884900,-1.8,2.8,0.575000,0.65,-4.8,1.1,-1.387500,-0.65,73.0,99.0,87.000000,86.5,0.8,40.2,17.841667,16.10,98.51,99.91,99.075833,98.840,-4.976663,0.686009,-1.884900,-1.513575,1,1,1994,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1994-01-02,-8.704167,-12.445833,74.416667,17.875000,28.387500,99.920000,-16.681818,15783.333333,-12.827363,-14.3,1.7,-8.704167,-9.90,-20.3,0.5,-12.445833,-13.90,60.0,92.0,74.416667,75.5,6.4,40.2,28.387500,25.00,98.77,100.51,99.920000,100.125,-19.165499,-0.338394,-12.827363,-14.393998,2,1,1994,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1994-01-03,-12.612500,-15.820833,77.250000,17.666667,10.404167,100.057083,-20.583333,18992.708333,-17.143932,-16.3,-10.3,-12.612500,-12.00,-23.2,-12.6,-15.820833,-15.00,55.0,91.0,77.250000,80.5,1.2,25.0,10.404167,8.00,99.41,100.47,100.057083,100.155,-21.317386,-14.555717,-17.143932,-16.423198,3,1,1994,16477.083333,NaN,NaN,NaN,NaN,NaN
3,3,1994-01-04,-9.833333,-13.570833,74.416667,25.791667,24.879167,98.675417,-18.541667,19080.708333,-14.183286,-11.5,-7.4,-9.833333,-9.95,-15.0,-11.5,-13.570833,-13.70,57.0,84.0,74.416667,76.0,9.7,40.2,24.879167,24.10,98.27,99.34,98.675417,98.610,-15.874948,-11.706660,-14.183286,-14.310177,4,1,1994,17952.250000,NaN,NaN,NaN,NaN,NaN
4,4,1994-01-05,-11.866667,-16.154167,70.791667,22.875000,27.033333,99.352083,-20.521739,19183.208333,-16.407695,-17.5,-7.2,-11.866667,-11.75,-20.8,-10.7,-16.154167,-17.00,56.0,84.0,70.791667,74.0,1.0,40.2,27.033333,25.00,98.44,100.37,99.352083,99.375,-22.394182,-11.256395,-16.407695,-16.464188,5,1,1994,19085.541667,NaN,NaN,NaN,NaN,NaN
5,5,1994-01-06,-16.000000,-18.533333,80.875000,22.250000,9.858333,100.021667,-25.541667,20335.791667,-20.744823,-18.2,-14.0,-16.000000,-15.75,-22.1,-16.2,-18.533333,-17.60,70.0,88.0,80.875000,83.0,1.0,25.0,9.858333,3.20,99.71,100.41,100.021667,100.025,-23.127172,-18.584225,-20.744823,-20.476987,6,1,1994,19533.236111,NaN,NaN,NaN,NaN,NaN
6,6,1994-01-07,-15.420833,-18.379167,78.208333,22.708333,15.858333,99.747083,-25.208333,20053.000000,-20.156319,-17.4,-13.7,-15.420833,-15.90,-21.0,-15.1,-18.379167,-19.00,66.0,90.0,78.208333,79.5,1.2,40.2,15.858333,12.90,99.47,99.97,99.747083,99.810,-22.253714,-18.192879,-20.156319,-20.628703,7,1,1994,19857.333333,2187.271689,-10.551786,5.584224,77.565476,5.262928
7,7,1994-01-08,-13.225000,-15.316667,84.333333,16.041667,10.654167,99.271667,-20.541667,18435.000000,-17.718462,-15.0,-9.5,-13.225000,-14.10,-17.8,-10.7,-15.316667,-16.45,78.0,92.0,84.333333,84.0,1.6,25.0,10.654167,9.70,98.97,99.81,99.271667,99.190,-19.704437,-13.544552,-17.718462,-18.662419,8,1,1994,19607.930556,1495.271012,-12.523214,2.684410,77.184524,4.508663
8,8,1994-01-09,-17.804167,-22.325000,68.083333,7.666667,31.295833,100.806667,-25.058824,18043.750000,-22.771558,-21.8,-12.2,-17.804167,-18.50,-26.3,-17.6,-22.325000,-22.40,56.0,76.0,68.083333,69.5,24.1,40.2,31.295833,25.00,99.91,101.71,100.806667,100.840,-26.945633,-17.004725,-22.771558,-23.458662,9,1,1994,18843.916667,815.105711,-13.823214,2.729764,76.279762,5.648137
9,9,1994-01-10,-14.858333,-18.841667,71.916667,10.750000,20.783333,101.655000,-20.904762,19717.916667,-19.501955,-24.8,-5.2,-14.858333,-15.60,-30.1,-9.1,-18.841667,-20.00,56.0,93.0,71.916667,71.5,4.0,25.0,20.783333,25.00,100.67,102.07,101.655000,101.810,-30.064700,-9.069644,-19.501955,-20.441878,10,1,1994,18732.222222,836.029569,-14.144048,2.695514,75.517857,5.851502


### Generating Lag features between one day and 7 day period

In [20]:
# df_clean['demand_lag_1'] = df_clean['daily_demand'].shift(1)
# df_clean['demand_lag_7'] = df_clean['daily_demand'].shift(7)


In [21]:
for lag in range(1,8):
  df_daily[f'demand_lag_{lag}'] = df_daily['daily_demand'].shift(lag)
  df_daily[f'demand_lag_{lag}'] = df_daily['daily_demand'].shift(lag)
  df_daily[f'rel_hum_lag_{lag}'] = df_daily['rel_hum'].shift(lag)
  df_daily[f'wind_speed_lag_{lag}'] = df_daily['wind_speed'].shift(lag)

### generating interactive features

In [22]:
df_daily['temp_humidity_interaction'] = df_daily['temp'] * df_daily['rel_hum']
df_daily['temp_wind_speed_interaction'] = df_daily['temp'] * df_daily['wind_speed']

In [23]:
# df_daily.drop(columns=['temp', 'dew_point_temp', 'rel_hum', 'visibility','press', 'hmdxx', ], inplace = True)

In [24]:
df_daily.head(30)

,index,datetime,temp,dew_point_temp,rel_hum,wind_speed,visibility,press,wind_chill,daily_demand,hmdxx,temp_min,temp_max,temp_mean,temp_median,dew_point_temp_min,dew_point_temp_max,dew_point_temp_mean,dew_point_temp_median,rel_hum_min,rel_hum_max,rel_hum_mean,rel_hum_median,visibility_min,visibility_max,visibility_mean,visibility_median,press_min,press_max,press_mean,press_median,hmdxx_min,hmdxx_max,hmdxx_mean,hmdxx_median,date,month,year,demand_rolling_mean_3,demand_rolling_std_7,temp_roll_mean,temp_roll_std,rel_hum_roll_mean,rel_hum_roll_std,demand_lag_1,rel_hum_lag_1,wind_speed_lag_1,demand_lag_2,rel_hum_lag_2,wind_speed_lag_2,demand_lag_3,rel_hum_lag_3,wind_speed_lag_3,demand_lag_4,rel_hum_lag_4,wind_speed_lag_4,demand_lag_5,rel_hum_lag_5,wind_speed_lag_5,demand_lag_6,rel_hum_lag_6,wind_speed_lag_6,demand_lag_7,rel_hum_lag_7,wind_speed_lag_7,temp_humidity_interaction,temp_wind_speed_interaction
0,0,1994-01-01,0.575000,-1.387500,87.000000,20.333333,17.841667,99.075833,-5.888889,14655.208333,-1.884900,-1.8,2.8,0.575000,0.65,-4.8,1.1,-1.387500,-0.65,73.0,99.0,87.000000,86.5,0.8,40.2,17.841667,16.10,98.51,99.91,99.075833,98.840,-4.976663,0.686009,-1.884900,-1.513575,1,1,1994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.025000,11.691667
1,1,1994-01-02,-8.704167,-12.445833,74.416667,17.875000,28.387500,99.920000,-16.681818,15783.333333,-12.827363,-14.3,1.7,-8.704167,-9.90,-20.3,0.5,-12.445833,-13.90,60.0,92.0,74.416667,75.5,6.4,40.2,28.387500,25.00,98.77,100.51,99.920000,100.125,-19.165499,-0.338394,-12.827363,-14.393998,2,1,1994,NaN,NaN,NaN,NaN,NaN,NaN,14655.208333,87.000000,20.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-647.735069,-155.586979
2,2,1994-01-03,-12.612500,-15.820833,77.250000,17.666667,10.404167,100.057083,-20.583333,18992.708333,-17.143932,-16.3,-10.3,-12.612500,-12.00,-23.2,-12.6,-15.820833,-15.00,55.0,91.0,77.250000,80.5,1.2,25.0,10.404167,8.00,99.41,100.47,100.057083,100.155,-21.317386,-14.555717,-17.143932,-16.423198,3,1,1994,16477.083333,NaN,NaN,NaN,NaN,NaN,15783.333333,74.416667,17.875000,14655.208333,87.000000,20.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-974.315625,-222.820833
3,3,1994-01-04,-9.833333,-13.570833,74.416667,25.791667,24.879167,98.675417,-18.541667,19080.708333,-14.183286,-11.5,-7.4,-9.833333,-9.95,-15.0,-11.5,-13.570833,-13.70,57.0,84.0,74.416667,76.0,9.7,40.2,24.879167,24.10,98.27,99.34,98.675417,98.610,-15.874948,-11.706660,-14.183286,-14.310177,4,1,1994,17952.250000,NaN,NaN,NaN,NaN,NaN,18992.708333,77.250000,17.666667,15783.333333,74.416667,17.875000,14655.208333,87.000000,20.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-731.763889,-253.618056
4,4,1994-01-05,-11.866667,-16.154167,70.791667,22.875000,27.033333,99.352083,-20.521739,19183.208333,-16.407695,-17.5,-7.2,-11.866667,-11.75,-20.8,-10.7,-16.154167,-17.00,56.0,84.0,70.791667,74.0,1.0,40.2,27.033333,25.00,98.44,100.37,99.352083,99.375,-22.394182,-11.256395,-16.407695,-16.464188,5,1,1994,19085.541667,NaN,NaN,NaN,NaN,NaN,19080.708333,74.416667,25.791667,18992.708333,77.250000,17.666667,15783.333333,74.416667,17.875000,14655.208333,87.000000,20.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-840.061111,-271.450000
5,5,1994-01-06,-16.000000,-18.533333,80.875000,22.250000,9.858333,100.021667,-25.541667,20335.791667,-20.744823,-18.2,-14.0,-16.000000,-15.75,-22.1,-16.2,-18.533333,-17.60,70.0,88.0,80.875000,83.0,1.0,25.0,9.858333,3.20,99.71,100.41,100.021667,100.025,-23.127172,-18.584225,-20.744823,-20.476987,6,1,1994,19533.236111,NaN,NaN,NaN,NaN,NaN,19183.208333,70.791667,22.875000,19080.708333,74.416667,25.791667,18992.708333,77.250000,17.666667,15783.333333,74.416667,17.875000,14655.208333,87.000000,20.333333,NaN,NaN,NaN,NaN,NaN,NaN,-1294.000000,-356.000000
6,6,1994-01-07,-15.420833,-18.379167,78.208333,22.708333,15.858333,99.747083,-25.208333,20053.000000,-20.156319,-17.4,-13.7,-15.420833,-15.90,-21.0,-15.1

In [25]:
# df_daily.dropna(inplace=True)

In [26]:
df_daily.head()

,index,datetime,temp,dew_point_temp,rel_hum,wind_speed,visibility,press,wind_chill,daily_demand,hmdxx,temp_min,temp_max,temp_mean,temp_median,dew_point_temp_min,dew_point_temp_max,dew_point_temp_mean,dew_point_temp_median,rel_hum_min,rel_hum_max,rel_hum_mean,rel_hum_median,visibility_min,visibility_max,visibility_mean,visibility_median,press_min,press_max,press_mean,press_median,hmdxx_min,hmdxx_max,hmdxx_mean,hmdxx_median,date,month,year,demand_rolling_mean_3,demand_rolling_std_7,temp_roll_mean,temp_roll_std,rel_hum_roll_mean,rel_hum_roll_std,demand_lag_1,rel_hum_lag_1,wind_speed_lag_1,demand_lag_2,rel_hum_lag_2,wind_speed_lag_2,demand_lag_3,rel_hum_lag_3,wind_speed_lag_3,demand_lag_4,rel_hum_lag_4,wind_speed_lag_4,demand_lag_5,rel_hum_lag_5,wind_speed_lag_5,demand_lag_6,rel_hum_lag_6,wind_speed_lag_6,demand_lag_7,rel_hum_lag_7,wind_speed_lag_7,temp_humidity_interaction,temp_wind_speed_interaction
0,0,1994-01-01,0.575000,-1.387500,87.000000,20.333333,17.841667,99.075833,-5.888889,14655.208333,-1.884900,-1.8,2.8,0.575000,0.65,-4.8,1.1,-1.387500,-0.65,73.0,99.0,87.000000,86.5,0.8,40.2,17.841667,16.1,98.51,99.91,99.075833,98.840,-4.976663,0.686009,-1.884900,-1.513575,1,1,1994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.025000,11.691667
1,1,1994-01-02,-8.704167,-12.445833,74.416667,17.875000,28.387500,99.920000,-16.681818,15783.333333,-12.827363,-14.3,1.7,-8.704167,-9.90,-20.3,0.5,-12.445833,-13.90,60.0,92.0,74.416667,75.5,6.4,40.2,28.387500,25.0,98.77,100.51,99.920000,100.125,-19.165499,-0.338394,-12.827363,-14.393998,2,1,1994,NaN,NaN,NaN,NaN,NaN,NaN,14655.208333,87.000000,20.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-647.735069,-155.586979
2,2,1994-01-03,-12.612500,-15.820833,77.250000,17.666667,10.404167,100.057083,-20.583333,18992.708333,-17.143932,-16.3,-10.3,-12.612500,-12.00,-23.2,-12.6,-15.820833,-15.00,55.0,91.0,77.250000,80.5,1.2,25.0,10.404167,8.0,99.41,100.47,100.057083,100.155,-21.317386,-14.555717,-17.143932,-16.423198,3,1,1994,16477.083333,NaN,NaN,NaN,NaN,NaN,15783.333333,74.416667,17.875000,14655.208333,87.000000,20.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-974.315625,-222.820833
3,3,1994-01-04,-9.833333,-13.570833,74.416667,25.791667,24.879167,98.675417,-18.541667,19080.708333,-14.183286,-11.5,-7.4,-9.833333,-9.95,-15.0,-11.5,-13.570833,-13.70,57.0,84.0,74.416667,76.0,9.7,40.2,24.879167,24.1,98.27,99.34,98.675417,98.610,-15.874948,-11.706660,-14.183286,-14.310177,4,1,1994,17952.250000,NaN,NaN,NaN,NaN,NaN,18992.708333,77.250000,17.666667,15783.333333,74.416667,17.875000,14655.208333,87.000000,20.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-731.763889,-253.618056
4,4,1994-01-05,-11.866667,-16.154167,70.791667,22.875000,27.033333,99.352083,-20.521739,19183.208333,-16.407695,-17.5,-7.2,-11.866667,-11.75,-20.8,-10.7,-16.154167,-17.00,56.0,84.0,70.791667,74.0,1.0,40.2,27.033333,25.0,98.44,100.37,99.352083,99.375,-22.394182,-11.256395,-16.407695,-16.464188,5,1,1994,19085.541667,NaN,NaN,NaN,NaN,NaN,19080.708333,74.416667,25.791667,18992.708333,77.250000,17.666667,15783.333333,74.416667,17.875000,14655.208333,87.0,20.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-840.061111,-271.450000


In [27]:
df_daily.to_csv('/Users/yashwanthkaruparthi/Developer/energy_demand/resrc/data/8-feat-eng/feat-lag.csv')